# Aoororachain - Perform QA Retrieval with Open Source LLM

Given that you already have your documents loaded into a vector database, then you can start query them with an Open Source LLM.

In [ ]:
require "aoororachain"

## Configuration

In [ ]:
Aoororachain.logger = Logger.new($stdout)
Aoororachain.log_level = Aoororachain::LEVEL_DEBUG

chroma_host = "http://localhost:8000"
collection_name = "prestavaro"

The model needs to be that same you used to embed your documents.

In [ ]:
model = Aoororachain::Embeddings::LocalPythonEmbedding::MODEL_INSTRUCTOR_L
device = "mps" # cuda or cpu

In [ ]:
embedder = Aoororachain::Embeddings::LocalPythonEmbedding.new(model:, device:)
vector_database = Aoororachain::VectorStores::Chroma.new(embedder: embedder, options: {host: chroma_host})

vector_database.from_index(collection_name)

retriever = Aoororachain::VectorStores::Retriever.new(vector_database)

# QA Retrieval with LLM

Start your LLM Server and set the host. Configure the chain.

In [5]:
llm = Aoororachain::Llms::LlamaServer.new("http://localhost:9292")
chain = Aoororachain::Chains::RetrievalQA.new(llm, retriever)

#<Aoororachain::Chains::RetrievalQA:0x000000010480ea80 @llm=#<Aoororachain::Llms::LlamaServer:0x000000010480edc8>, @retriever=#<Aoororachain::VectorStores::Retriever:0x00000001047e4a78 @vector_store=#<Aoororachain::VectorStores::Chroma:0x00000001047ef0b8 @embedder=#<Aoororachain::Embeddings::LocalPythonEmbedding:0x00000001047c2748 @model="hkunlp/instructor-large", @device="mps">, @store=#<Chroma::Resources::Collection:0x00000001047e4b18 @id="b3f2ff9f-c359-45e9-9f8f-d62c99c465fd", @name="ley-fintech", @metadata={"embedder"=>"Aoororachain::Embeddings::LocalPythonEmbedding : hkunlp/instructor-large : mps"}>>, @search_type=:similarity, @results=3>, @type=:stuff>

In [ ]:
response = chain.complete(prompt: "¿Como puedo pagar?")

In [7]:
response

{:response=>"  Asistente: Una institución de tecnología financiera (ITF) es un tipo de entidad financiera que utiliza tecnologías avanzadas para ofrecer servicios financieros, como préstamos, inversiones y gestión de activos. Estas instituciones se enfocan en la innovación y el uso de herramientas digitales para mejorar la eficiencia y accesibilidad del sector financiero.", :sources=>[{"source"=>"./files/LRITF_200521.pdf", "pages"=>71, "page"=>6}, {"source"=>"./files/LRITF_200521.pdf", "pages"=>71, "page"=>55}, {"source"=>"./files/LRITF_200521.pdf", "pages"=>71, "page"=>11}]}